In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os

In [27]:
colunas_rename = {
    "pos_FW": "Pos Atacante",
    "last_time_played": "Tempo jogado no último jogo (min)",
    "last_touches": "Toques no último jogo",
    "last_xg": "xG no último jogo",
    "last_completed_passes_perc": "Percentual de passes completados no último jogo",
    "player_age": "Idade do jogador",
    "days_from_last_game": "Dias desde o último jogo",
    "Capacity": "Capacidade do estádio",
    "travel_distance": "Distância viajada para o jogo (Km)",
    "pos_LW": "Pos Lateral Esquerdo",
    "last_xag": "xAG no último jogo",
    "tmax": "Temperatura máxima",
    "wspd": "Velocidade do vento média",
    "Wk": "Semana do campeonato",
    "pos_CM": "Pos Meio-Campista",
    "tavg": "Temperatura média",
    "wpgt": "Rajada de vento máxima",
    "tmin": "Temperatura mínima",
    "pos_RW": "Pos Lateral Direito",
    "played_at_home_False": "Jogar fora de casa",
    "pos_AM": "Pos Atacante Meio-Campista",
    "last_shots_on_target": "Chutes a gol no último jogo",
    "last_goals": "Gols no último jogo",
    "last_result_win": "Vitória no útimo jogo",
    "prcp": "Precipitação (mm)",
    "last_result_draw": "Empate no último jogo",
    "last_result_lose": "Derrota no último jogo",
    "pos_RM": "Pos Meio-Campista Direito",
    "last_yellow_cards": "Cartões amarelo no último jogo",
    "pos_LM": "Pos Meio-Campista Esquerdo",
    "snow": "Neve",
    "played_at_home_True": "Jogar em casa",
}

In [23]:
def categorical_to_one_hot_encoding(dataset, categorical_columns):
    """
    Convert categorical columns to one hot encoding.

    Parameters
    ----------
    dataset : pd.DataFrame
        The dataset to be converted.
    categorical_columns : list
        The list of categorical columns.

    Returns
    -------
    dataset_converted : pd.DataFrame
        The converted dataset.
    """
    dataset_converted = dataset.copy()
    for column in categorical_columns:
        dataset_converted = pd.concat([dataset_converted, pd.get_dummies(dataset_converted[column], prefix=column, dtype=float)], axis=1)
        dataset_converted.drop(column, axis=1, inplace=True)

    return dataset_converted

def load_dataset(path):
    dataset = pd.read_parquet(path)
    dataset.drop(["Season_End_Year", "Team", "Away"], axis=1, inplace=True)
    return categorical_to_one_hot_encoding(dataset, categorical_columns=["pos", "played_at_home", "last_result"])

In [28]:
df = load_dataset("data/final_datasets/dataset_xg.parquet")

In [35]:
def most_frequent(v):
    most = dict()
    for x in v:
        if x not in most:
            most[x] = 0
        most[x] += 1
    return sorted(most.items(), key = lambda x: x[1], reverse = True)

In [43]:
players = [("Roberto Firmino", "green"), ("Mohamed Salah","red")]

In [45]:
targets = {"XG": "data/final_datasets/dataset_xg.parquet",
           "XT": "data/final_datasets/dataset_xt.parquet",
           "VAEP": "data/final_datasets/dataset_vaep.parquet"}


for target, path in targets.items():
    df_og = load_dataset(path)
    for player,c in players:
        df = df_og[df_og["player"] == player]
        for k,v in colunas_rename.items():
            if k not in df.columns: continue
            plt.scatter(df[k], df["target"], color=c)
            plt.xlabel(v)
            plt.ylabel(target)
            plt.title(f"Relação entre {v} e {target} - {player}")
            
            os.makedirs(f"plots/{player}/{target}", exist_ok = True)
            plt.savefig(f"plots/{player}/{target}/{v}.png")
            plt.close()